In [ ]:
import numpy as np
import pandas as pd

GRID = 'grid'
MG = 'mg'
SHS = 'shs'
ELECTRIFICATION_OPTIONS = [GRID, MG, SHS]
BAU_SCENARIO = 'bau'
SE4ALL_SCENARIO = 'uea'
SE4ALL_FLEX_SCENARIO = 'se4all_shift'
PROG_SCENARIO = 'prog'
SCENARIOS = [BAU_SCENARIO, SE4ALL_SCENARIO, PROG_SCENARIO]

# Names for display
SCENARIOS_DICT = {
    BAU_SCENARIO: 'BaU',
    SE4ALL_SCENARIO: 'uEA',
    PROG_SCENARIO: 'prOG',
}

ELECTRIFICATION_DICT = {
    GRID: 'Grid',
    MG: 'Mini Grid',
    SHS: 'Solar Home System'
}

# column names of the exogenous results
ENDO_POP_GET = ['endo_pop_get_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
POP_GET = ['pop_get_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_GET = ['hh_get_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_CAP = ['hh_%s_capacity' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_SCN2 = ['hh_cap_scn2_%s_capacity' % opt for opt in ELECTRIFICATION_OPTIONS]
INVEST = ['%s_investment_cost' % opt for opt in ELECTRIFICATION_OPTIONS]
INVEST_CAP = ['tier_capped_%s_investment_cost' % opt for opt in ELECTRIFICATION_OPTIONS]
GHG = ['ghg_%s_cumul' % opt for opt in ELECTRIFICATION_OPTIONS] + ['ghg_no_access_cumul']
GHG_ER = ['ghg_%s_ER_cumul' % opt for opt in ELECTRIFICATION_OPTIONS] \
         + ['ghg_no_access_ER_cumul']
GHG_CAP = ['tier_capped_ghg_%s_cumul' % opt for opt in ELECTRIFICATION_OPTIONS] \
          + ['tier_capped_ghg_no_access_cumul']
GHG_CAP_ER = ['tier_capped_ghg_%s_ER_cumul' % opt for opt in ELECTRIFICATION_OPTIONS] \
             + ['tier_capped_ghg_no_access_ER_cumul']
GHG_ALL = GHG + GHG_ER + GHG_CAP + GHG_CAP_ER \
          + ['ghg_tot_cumul', 'tier_capped_ghg_tot_cumul'] \
          + ['ghg_tot_ER_cumul', 'tier_capped_ghg_tot_ER_cumul'] \
    + ['ghg_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS] \
    + ['tier_capped_ghg_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
EXO_RESULTS = POP_GET + HH_GET + HH_CAP + HH_SCN2 + INVEST + INVEST_CAP + GHG_ALL

# source http://www.worldbank.org/content/dam/Worldbank/Topics/Energy%20and%20Extract/
# Beyond_Connections_Energy_Access_Redefined_Exec_ESMAP_2015.pdf
MIN_TIER_LEVEL = 3
MIN_RATED_CAPACITY = {1: 3, 2: 50, 3: 200, 4: 800, 5: 2000}  # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {1: 4.5, 2: 73, 3: 365, 4: 1250, 5: 3000}  # index is TIER level [kWh/a]
RATIO_CAP_CONSUMPTION = {}

# Investment Cost Source: Arranz and Worldbank,
# BENCHMARKING STUDY OF SOLAR PV MINIGRIDS INVESTMENT COSTS, 2017 (Jabref)
# unit is USD per household
MEDIAN_INVESTMENT_COST = {1: 742, 2: 1273, 3: 2516, 4: 5277, 5: 5492}

# drives for the socio-economic model
IMPACT_FACTORS = pd.DataFrame(
    {
        MG: [3, 13. / 6, 19. / 6, 3.25, 11. / 3],
        SHS: [23. / 12, 4.5, 37. / 12, 17. / 6, 41. / 12],
        'labels': [
            'high_gdp',
            'high_mobile_money',
            'high_ease_doing_business',
            'low_corruption',
            'high_grid_weakness'
        ]
    }
)
IMPACT_FACTORS = IMPACT_FACTORS.set_index('labels')

MENTI_DRIVES = ['gdp', 'mobile_money', 'ease_doing_business', 'corruption', 'weak_grid']

# $RT_shift_factors.$P$2
WEIGHT_MENTIS = 0.2
# -->WEIGHT_GRID = 0.8 ($RT_shift_factors.$O$2)  and  WEIGHT_GRID = 1 - WEIGHT_MENTIS
RISE_INDICES = ['rise_%s' % opt for opt in ELECTRIFICATION_OPTIONS]
SHIFT_MENTI = ['shift_menti_mg', 'shift_menti_shs']

BASIC_ROWS = [
    'People share',
    'People (k)',
    'HH (k)',
    'HH cap. (MW)',
    'HH cap. (MW) (TIER + 1)',
    'Investment MUSD',
    'Investment (TIER + 1) MUSD',
]
# labels of the columns of the result tables
LABEL_COLUMNS = ELECTRIFICATION_DICT.copy()
# a column for the row labels
LABEL_COLUMNS['labels'] = ''
LABEL_COLUMNS['total'] = 'Total'
BASIC_COLUMNS_ID = ['labels'] + ELECTRIFICATION_OPTIONS + ['total']
GHG_COLUMNS_ID = ['labels'] + ELECTRIFICATION_OPTIONS + ['total']
COMPARE_COLUMNS_ID = ['labels']
for opt in ELECTRIFICATION_OPTIONS + ['total']:
    COMPARE_COLUMNS_ID.append(opt)
    COMPARE_COLUMNS_ID.append('comp_{}'.format(opt))


In [ ]:
# Save the results for a specific country for each scenario
from data.data_preparation import compute_ndc_results_from_raw_data
iso = 'NGA'

for sce in SCENARIOS:
    df = compute_ndc_results_from_raw_data(sce, MIN_TIER_LEVEL)
    df = df.loc[df.country_iso == iso]
    df = df[EXO_RESULTS].transpose()
    print(sce)
    print(df)
    df.to_csv('NDC_results_{}_{}.csv'.format(sce, iso))

In [ ]:
# Save the cumulated results for each scenario
from data.data_preparation import compute_ndc_results_from_raw_data, EXO_RESULTS
for sce in SCENARIOS:
    df = compute_ndc_results_from_raw_data(sce, MIN_TIER_LEVEL)
    
    df = df[EXO_RESULTS].sum(axis=0)
    print(sce)
    print(df)
    df.to_csv('NDC_results_{}.csv'.format(sce))

In [ ]:
from data.data_preparation import SHS_SALES_VOLUMES, SHS_POWER_CATEGORIES, SHS_AVERAGE_INVESTMENT_COST

In [ ]:
SHS_SALES_VOLUMES

In [ ]:
SHS_AVERAGE_INVESTMENT_COST
shs_costs = pd.read_csv('data/shs_power_investment_cost.csv', comment='#')
shs_costs

In [ ]:
# Save the results for each scenario for tests
from data.data_preparation import compute_ndc_results_from_raw_data, EXO_RESULTS
for sce in SCENARIOS:
    df = compute_ndc_results_from_raw_data(sce, MIN_TIER_LEVEL)
    df = df[EXO_RESULTS + ['country', 'country_iso']]
    df.to_csv('tests/data/results_test_comparison_{}.csv'.format(sce))


## Test TIER level attribution

In [ ]:
from data_preparation import _find_tier_level

for hh_cons in np.array([1, 50, 100, 400, 2000, 5000]):
    print(_find_tier_level(hh_cons, 2))

In [ ]:
from data_preparation import _slope_capacity_vs_yearly_consumption
RATIO_CAP_CONSUMPTION = {}
TIER_LEVELS = [1, 2, 3, 4, 5]
for tier_lvl in [1, 2, 3, 4]:
    RATIO_CAP_CONSUMPTION[tier_lvl] = _slope_capacity_vs_yearly_consumption(tier_lvl)
RATIO_CAP_CONSUMPTION

In [ ]:
MIN_RATED_CAPACITY = {1: 3, 2: 50, 3: 200, 4: 800, 5: 2000}  # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {1: 4.5, 2: 73, 3: 365, 4: 1250, 5: 3000}  # index is TIER level [kWh/a]

In [ ]:
for tier_lvl in TIER_LEVELS:
    print(MIN_RATED_CAPACITY[tier_lvl] / MIN_ANNUAL_CONSUMPTION[tier_lvl])

In [ ]:
cap = [MIN_RATED_CAPACITY[i] for i in TIER_LEVELS]
df = pd.DataFrame(cap)
df[0].plot()

In [ ]:
cons = [MIN_ANNUAL_CONSUMPTION[i] for i in TIER_LEVELS]
df = pd.DataFrame(cons)
df[0].plot()

In [ ]:
from data_preparation import IMPACT_FACTORS
IMPACT_FACTORS.index.to_list()

In [ ]:
for opt in [MG, SHS]:
    for input_name in IMPACT_FACTORS.index.to_list():
        #print("Input('impact-{}-{}-input', 'value'),".format(opt, input_name.replace('_', '-')))
        print('impact_{}_{},'.format(opt, input_name.replace('high', '')))



# Tests of the model

In [ ]:
from data.data_preparation import compute_ndc_results_from_raw_data
SCENARIOS_DATA = {
    sce: compute_ndc_results_from_raw_data(sce, MIN_TIER_LEVEL).to_json() for sce in SCENARIOS
}

In [ ]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO])

In [ ]:
a = se_df[POP_GET ]
a['summe'] = a.sum(axis=1)
a['tot'] = se_df['pop_newly_electrified_2030']
a['diffe'] = a.tot.values - a.summe.values
a.loc[a.diffe > 5e-9]
a[POP_GET] = a[POP_GET].div(se_df.pop_newly_electrified_2030, axis=0)
a['summe'] = a[POP_GET].sum(axis=1)
a

In [ ]:
df

In [ ]:
xls_bau = pd.read_csv('data/xls_bau.csv', float_precision='high')
xls_se = pd.read_csv('data/xls_se.csv', float_precision='high')
xls_prog = pd.read_csv('data/xls_prog.csv', float_precision='high')

invest_bau = pd.read_csv('data/invest_bau.csv', float_precision='high')
invest_se = pd.read_csv('data/invest_se.csv', float_precision='high')
invest_prog = pd.read_csv('data/invest_prog.csv', float_precision='high')

## Test RISE new model

In [ ]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
xls_se = pd.read_csv('NDC_full_results_old_rise_model_se4all.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = EXO_RESULTS

df_diff = xls_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=1):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff[INVEST].loc[l]

df_diff[COMP_COLS].to_csv('se4all_old_model_minus_updated.csv')
xls_se[COMP_COLS].to_csv('se4all_old_model.csv')
se_df[COMP_COLS].to_csv('se4all_updated_model.csv')


## Test Exogenous results

### BaU

In [ ]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
xls_bau = pd.read_csv('data/xls_bau.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = POP_GET + HH_GET + HH_CAP + HH_SCN2

df_diff = xls_bau[COMP_COLS] - bau_df[COMP_COLS]

def highlight_mismatch(col, eps=0.2):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

### SE4ALL

In [ ]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
xls_se = pd.read_csv('data/xls_uea.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = POP_GET + HH_GET + HH_CAP + HH_SCN2

df_diff = xls_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=0.001):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]


### prOG

In [ ]:
xls_prog = pd.read_csv('data/xls_prog.csv', float_precision='high')
prog_df = pd.read_json(SCENARIOS_DATA[PROG_SCENARIO]).set_index('country_iso').sort_index(ascending=True)


COMP_COLS = POP_GET + HH_GET + HH_CAP + HH_SCN2

df_diff = xls_prog[COMP_COLS] - prog_df[COMP_COLS]

def highlight_mismatch(col, eps=0.2):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

## Test GHG

### BaU

In [ ]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_bau = pd.read_csv('data/ghg_bau.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_bau[COMP_COLS] - bau_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

### SE4All

In [ ]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_se = pd.read_csv('data/ghg_se.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

## Saved from BaU

In [ ]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

saved_se_df = bau_df[COMP_COLS] - se_df[COMP_COLS]


### prOG

In [ ]:
prog_df = pd.read_json(SCENARIOS_DATA[PROG_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_prog = pd.read_csv('data/ghg_prog.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_prog[COMP_COLS] - prog_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

## Test Investment cost

### BaU

In [ ]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_bau = pd.read_csv('data/invest_bau.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_bau[COMP_COLS] - bau_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

### SE4All

In [ ]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_se = pd.read_csv('data/invest_se.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=20):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

### prOG

In [ ]:
prog_df = pd.read_json(SCENARIOS_DATA[PROG_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_prog = pd.read_csv('data/invest_prog.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_prog[COMP_COLS] - prog_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

# Rise shifts for electrification option

$i \in $ (grid, mg, shs)

$N_i$ : population getting option $i$ in 2030

$R_i$ : RISE score for the option $i$

$\delta_{ij} = R_i - R_j$

$\Delta_i = \frac{\sum_j \delta_ij}{\sum R_k} $

$\Delta N_i = \Delta_i (\sum_j N_j)$

Constraint is that $\sum_j \Delta N_j$

## Sarah PF Model
$R_i \neq 0$, $\forall i$

$N_i - \Delta N_i < 0$,  for some $i$

if $N_i - \Delta N_i < 0$,  for one $i$ ([90,10,45], [30, 40, 30]), then we take $\Delta N_i = N_i \Delta_i < 0$ for this $i$ and we split this value between the $j$ for which $N_j - \Delta N_j > 0$, with the weight $\frac{R_j}{\sum_{k \neq i} R_k}$. That way we don't over-penalize

if $N_i - \Delta N_i < 0$,  for two $i$ ([90,10,15], [30, 40, 5]), then we take $\Delta N_i = N_i \Delta_i < 0$ for these $i$ and we add their  absolute sum to the $j$ for which $N_j - \Delta N_j > 0$. That way we don't over-penalize


The case ([90,10,45], [30, 700, 30]) penalize the R3 = 45 more than the R2=20 which is wrong, this is a limiting case to explore, maybe the solution would be to still penalize according to the initial $\Delta_{ij}$ (ie if $\Delta_{ij}$ < 0), then one must nevertheless be penalized, although it might be small and not have the penalty of the $i$ for which $N_i - \Delta N_i < 0$ redistributed equally among the remaining $j$

Maybe we want to cap the maximum value that can be given to a $N_i$ to be no larger than $N_i \Delta_i$, maybe not ?
between [90,5,55], [30, 600, 30] and [90,1,55], [30, 600, 30], the reward is dispoportionate in favor of R1 in first case

In [ ]:
set(1,2)

In [ ]:
import logging

logging.basicConfig(level=logging.ERROR)

def compute_rise_shifts_spfm(rise, pop_get, opt, flag=''):
    df = pd.DataFrame(
        data=[rise, pop_get, [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
        columns=ELECTRIFICATION_OPTIONS)

    please_print = True

    if df.iloc[0].sum() == 0:
        df.iloc[6] = df.iloc[0]
    else:
        df.iloc[2] = df.iloc[1] / df.iloc[1].sum()
        df.iloc[3, 0] = (df.iloc[0].grid - df.iloc[0].mg) + (df.iloc[0].grid - df.iloc[0].shs)
        df.iloc[3, 1] = (df.iloc[0].mg - df.iloc[0].grid) + (df.iloc[0].mg - df.iloc[0].shs)
        df.iloc[3, 2] = (df.iloc[0].shs - df.iloc[0].mg) + (df.iloc[0].shs - df.iloc[0].grid)

        df.iloc[3] = df.iloc[3] / df.iloc[0].sum()

        for j in range(3):
            df.iloc[4, j] = df.iloc[1].sum() * df.iloc[3, j]
            if df.iloc[1, j] != 0:
                df.iloc[5, j] = df.iloc[4, j] / df.iloc[1, j]
            else:
                df.iloc[5, j] = np.nan

        df.iloc[6] = df.iloc[1] + df.iloc[4]
        diff = df.iloc[6].values
        # logging.debug(diff)
        diff = diff[diff < 0]
        if len(diff) == 2:
            logging.debug('There are two differences smaller than 0')
            diff = df.iloc[6].values
            diff = diff[diff > 0]
            idx = df.iloc[6].to_list().index(diff[0])

            eps = 0
            for i in range(3):
                if i != idx:
                    df.iloc[6, i] = df.iloc[1, i] * df.iloc[3, i]
                    eps = eps + np.abs(df.iloc[6, i])
            df.iloc[6, idx] = eps

        elif len(diff) == 1:
            logging.debug('one difference is smaller than 0')
            idx = df.iloc[6].to_list().index(diff[0])
            norm = 0
            idx2 = None
            # find out if there is another penalized case
            for i in range(3):
                if i != idx:
                    if df.iloc[3, i] < 0:
                        idx2 = i
                    norm = norm + df.iloc[3, i]

            if idx2 is None:
                logging.debug('the difference will be fully split between the two other case')
                # we take N_i Delta_i away from N_i and split it amongst the remaining
                # options
                eps = df.iloc[1, idx] * df.iloc[3, idx]
                for i in range(3):
                    if i == idx:
                        df.iloc[6, i] = eps
                    else:
                        # weight Delta_i / sum(Delta_j, with j !=idx)
                        df.iloc[6, i] = np.abs(eps) * df.iloc[3, i] / norm
            else:
                logging.debug('one of the remaining should have a penalty')

                over_penalty = []
                for j in range(3):
                    if df.iloc[4, j] <= df.iloc[1, j] * df.iloc[3, j]:
                        over_penalty.append(j)
                        df.iloc[6, j] = df.iloc[1, j] * df.iloc[3, j]
                idx_not_penalised = list(set(range(3)) - set(over_penalty))[0]

                df.iloc[6, idx_not_penalised] = np.abs(df.iloc[6, over_penalty].sum())

                # eps = df.iloc[1, idx] * df.iloc[3, idx]

                ## this one cannot be larger that its population
                # df.iloc[6, idx] = eps
                ## this one becomes a part of the population from above, which compensates
                ## a bit the penalty
                # df.iloc[6, idx2] = df.iloc[4, idx2] + np.abs(eps) * df.iloc[
                #    0, idx2] / norm
                ## the highest score receives the penalty
                # for i in range(3):
                #    if i != idx and i != idx2:
                #        #logging.debug(i)
                #        df.iloc[6, i] = np.abs(df.iloc[4, idx2]) + np.abs(eps) * \
                #                        df.iloc[0, i] / norm
        elif len(df) == 3:
            logging.debug('error, all differences are negative')
        else:
            logging.debug('no difference is smaller than 0')

            # check if any Delta_i < 0
            diff = df.iloc[3].values
            diff = diff[diff < 0]
            if len(diff) == 1:
                logging.debug('Only one should be penalized')
                logging.debug('The difference will be fully split between the two other case')
                idx = df.iloc[3].to_list().index(diff[0])
                norm = 0
                for i in range(3):
                    if i != idx:
                        # the sum of the Delta_i of each non-penalized
                        norm = norm + df.iloc[3, i]

                eps = df.iloc[1, idx] * df.iloc[3, idx]
                for i in range(3):
                    if i == idx:
                        df.iloc[6, i] = eps
                    else:
                        # weight Delta_i / sum(Delta_j, with j !=idx)
                        df.iloc[6, i] = np.abs(eps) * df.iloc[3, i] / norm

                        # ----
            elif len(diff) == 2:
                over_penalty = []
                for j in range(3):
                    if df.iloc[4, j] <= df.iloc[1, j] * df.iloc[3, j]:
                        over_penalty.append(j)
                        df.iloc[6, j] = df.iloc[1, j] * df.iloc[3, j]
                idx_not_penalised = list(set(range(3)) - set(over_penalty))[0]

                df.iloc[6, idx_not_penalised] = np.abs(df.iloc[6, over_penalty].sum())

                # logging.debug(over_penalty)
                logging.debug('Two should be penalized')
            elif len(diff) == 3:
                logging.debug('error, all deltas are negative')
            else:
                # case when all RISE are equal
                logging.debug('error, all deltas are positive or zero')
                df.iloc[6] = df.iloc[4]
                please_print = True

        df.iloc[7] = df.iloc[6] + df.iloc[1]
        for i in range(3):
            if df.iloc[1, i] != 0:
                df.iloc[5, i] = df.iloc[6, i] / df.iloc[1, i]
            else:
                df.iloc[5, i] = np.nan

    if df.iloc[6].sum() > 1e-6:
        logging.debug(
            'Error ({}): the sum of the shifts ({}) is not equal to zero!'.format(
                flag,
                df.iloc[6].sum(),
            )
        )
        please_print = True

    if please_print:
        logging.debug(flag)
        df['sum'] = df.sum(axis=1)

        df['labels'] = ['R_i', 'N_i', 'n_i', 'Delta_i', 'Delta N_i', 'Delta N_i / N_i (case 2)',
                        'Delta N_i (case 2)', 'Delta N_i + N_i (case 2)']
        # logging.debug(df)
        # logging.debug()
        # logging.debug()
    return df  # df.iloc[6, ELECTRIFICATION_OPTIONS.index(opt)]

vals=np.arange(1,100,1)
#vals=[13]
y = {'grid_vs_x':[], 'mg_vs_x': [], 'shs_vs_x': []}
for x in vals:
    pass
    #a = compute_rise_shifts([16.67,  35,  x], [3.9198e+07,  1.75457e+07,  4.09613e+07], 0)
    # [16.67,  35,  22.22]
    # [7.62143e+06,  5.86826e+07,  3.1401e+07]
    #y['grid_vs_x'].append(a.iloc[7, 0])
    #y['mg_vs_x'].append(a.iloc[7, 1])
    #y['shs_vs_x'].append(a.iloc[7, 2])


## Philipp Rise score model

### Inputs

$i \in $ (grid, mg, shs)

$N_i$ : predicted population getting option $i$ in 2030

$R_i$ : RISE score for the option $i$

### Definitions

$\delta_{ij} = R_i - R_j$

$\Delta N_i$ : what gets added or removed from $N_i$

$\Delta N_{ij}$ : what is taken from $N_i$ and is transferred to $N_j$

Constraint is that $\sum_j \Delta N_j$

### Calculation of $\Delta N_i$ in case $\delta_{ij} \neq 0$ $\forall i,j$

Find the minimum RISE score and compute the penalty:

$R_{n} = \min(\lbrace R_i \rbrace)$

$\Delta_n = (\max(\lbrace R_i \rbrace) - R_k) / 100$

$\Delta N_n = - N_n \Delta_n$

$\Delta N_{ni} = |\Delta N_n| \frac{\delta_{in}}{\sum_{j}\delta_{jn}}$ $\forall i \neq n$

Then we consider the set $\lbrace R_i \rbrace_{i \neq n}$, i.e the next minium score, and compute the penalty:

$R_{m} = \max(\lbrace R_i \rbrace_{i \neq n})$ (then index $p$ for the last index)

$\Delta N_{pm} =  N_p \frac{\delta_{mp}}{100}$

$\Delta N_{p} = \Delta N_{np} - \Delta N_{pm}$

$\Delta N_{m} = \Delta N_{nm} + \Delta N_{pm}$



In [ ]:
def compute_rise_shifts_philipp_model(rise, pop_get, opt, flag=''):
    df = pd.DataFrame(
        data=[rise, pop_get, [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
        columns=ELECTRIFICATION_OPTIONS)

    n = ELECTRIFICATION_OPTIONS.index(df.iloc[0].idxmin())
    R_n = df.iloc[0].min() 
    
    m = ELECTRIFICATION_OPTIONS.index(df.iloc[0].idxmax())
    R_m = df.iloc[0].max()
    
    p = list(set(range(3)) - set([m,n]))[0]
    R_p = df.iloc[0, p]
    
    if R_n == R_p and R_p == R_m:
        df.iloc[4] = 0
    else:
    
        # calulate n_i
        df.iloc[2] = df.iloc[1]/df.iloc[1].sum()

        Delta_n = (R_m - R_n)/(100)

        df.iloc[4, n] = - df.iloc[1, n] * Delta_n 

        norm = 0
        for j in [m, p]:
            delta_jn = (df.iloc[0, j] - df.iloc[0, n])
            norm = norm + delta_jn
            df.iloc[4, j] = np.abs(df.iloc[4, n]) * delta_jn 

        df.iloc[4, [m, p]] = df.iloc[4, [m, p]] / norm

        #--------------------------------------------------------

        DeltaN_pm = df.iloc[1, p] * (R_m - R_p) / (100)

        df.iloc[4, p] = df.iloc[4, p] - DeltaN_pm
        df.iloc[4, m] = df.iloc[4, m] + DeltaN_pm

    df.iloc[7] = df.iloc[4] + df.iloc[1]
    for i in range(3):
        df.iloc[5, i] = df.iloc[4, i] / df.iloc[1, i]

    df['sum'] = df.sum(axis=1)

    df['labels'] = ['R_i', 'N_i', 'n_i', '', 'Delta N_i', 'Delta N_i / N_i',
                        '', 'Delta N_i + N_i']

    if df.iloc[4, 3] > 1e-6:
        logging.error(
            'Error ({}): the sum of the shifts ({}) is not equal to zero!'.format(
                flag,
                df.iloc[6].sum(),
            )
        )

    return df
    #return df.iloc[6, ELECTRIFICATION_OPTIONS.index(opt)]

df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso')
  
# BDI [1.378121e+07, 2.418979e+05, 1.958563e+05]
a = compute_rise_shifts_philipp_model([20, 50, 50], [1e+07, 1e+05, 1e+05], 0)
# [16.67,  35,  22.22]
# [7.62143e+06,  5.86826e+07,  3.1401e+07]
df[ENDO_POP_GET]
a

# Models comparison

In [ ]:
y_vals = {'sarah_pf_model': {}, 'philipp_model': {}}

models = {'sarah_pf_model': compute_rise_shifts, 'philipp_model': compute_rise_shifts_philipp_model}
          
folder = {'sarah_pf_model': 'access_rise_model', 'philipp_model': 'access_rise_philipp_model'}

In [ ]:
# take picture of changing the values of one RISE at the time for each electrification option and each country
import matplotlib.pyplot as plt

def produce_fom(model_id):
    rise_shift_model = models[model_id]
    df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso')
    for iso in df.index:
        print(iso)
        endo = df.loc[iso, ENDO_POP_GET].values
        rise = df.loc[iso, RISE_INDICES].values
        vals=np.arange(0,101,1)
        y_vals[model_id][iso] = {}
        for i, opt in enumerate(ELECTRIFICATION_OPTIONS):
            #vals=[13]
            rises = np.vstack([rise for i in vals])
            rises[:, i] = vals
            y = []
            for r in rises:
                a = rise_shift_model(r, endo, 0, iso)
                y.append(a.iloc[7,0:3].values)
            y = np.vstack(y)
            
            y_vals[model_id][iso][opt] = y
            
            for j, y_i in enumerate(np.transpose(y)):
                opt2 = ELECTRIFICATION_OPTIONS[j]
                plt.plot(vals,y_i, label='%s_vary_%s' % (opt2, opt))
            plt.plot(vals, y.sum(axis=1))
            plt.xlabel('RISE-%s' %opt)
            plt.ylabel('Pop')
            plt.title('Rise = %s -- %s -- n=%s'%(str(list(rise)), iso, ['%.2f' % i  for i in endo/endo.sum()]))
            plt.legend()
            plt.savefig('%s/%s_vary_%s.png'%(folder[model_id], iso, opt))
            plt.clf()
            
model_ids = ['philipp_model', 'sarah_pf_model']

for model_id in model_ids:
    produce_fom(model_id)

In [ ]:
# Save plots of the model comparison in compare_models folder

df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso')
vals=np.arange(0,101,1)
colors = ['#7030a0', '#5b9bd5', '#ed7d31']
model_ids = ['philipp_model', 'sarah_pf_model']

for iso in df.index:
    print(iso)
    endo = df.loc[iso, ENDO_POP_GET].values
    rise = df.loc[iso, RISE_INDICES].values
    for i, opt in enumerate(ELECTRIFICATION_OPTIONS):
        for sg, model_id, model_short in zip(['-','--'], model_ids, ['pm', 'spfm']):
            y = y_vals[model_id][iso][opt]
            for j, y_i in enumerate(np.transpose(y)):
                opt2 = ELECTRIFICATION_OPTIONS[j]
                plt.plot(vals,y_i, sg, color=colors[j], label='%s_%s_vary_%s' % (model_short, opt2, opt))
        #plt.plot(vals, y.sum(axis=1))
        plt.xlabel('RISE-%s' %opt)
        plt.ylabel('Pop')
        plt.title('Rise = %s -- %s -- n=%s'%(str(list(rise)), iso, ['%.2f' % i  for i in endo/endo.sum()]))
        plt.legend()
        plt.savefig('%s/%s_vary_%s.png'%('compare_models', iso, opt))
        plt.clf()